In [1]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #

In [2]:
#API key
api_key = "RGAPI-aa3b58a4-5012-4ae0-9699-6f9083e6f763"
gameName = "Slylio" #username
tagLine = "EUW" #suffixe

In [3]:
#Player unviveraly unique identifier
requestPUUID = requests.get(f"https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}")
print(requestPUUID)
puuid = requestPUUID.json()["puuid"] 
print(puuid)

<Response [200]>
nICwUFHOH_TsQczqTIDUQioPzLXiN3fytgCmRxKhbq0puAX8nqzIEWAsPLP2prR7H3bQF3DOvJ4iyg


In [4]:
#Match history from PUUID 
nb_of_games = 2
games = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={nb_of_games}&api_key={api_key}")
for game in games.json():
    print(game)

EUW1_6835115969
EUW1_6835054292


In [5]:
#Extract matchs data from match ids
matchs = games.json()
matchs_data=[]
data = pd.DataFrame()
cout=0
for matchId in matchs:
    cout+=1
    print("Match n°",cout,"/",nb_of_games,"\r",end="")
    match_data = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}")
    matchs_data.append(match_data.json())
    


In [6]:
matchs_data[0]["info"]["participants"][0]["perks"]["styles"][0]
print(type(matchs_data[0]["info"]["participants"][0]["perks"]["styles"][0]))
matchs_data[0]["info"]["participants"][0]["perks"]

<class 'dict'>


{'statPerks': {'defense': 5001, 'flex': 5008, 'offense': 5008},
 'styles': [{'description': 'primaryStyle',
   'selections': [{'perk': 8230, 'var1': 18, 'var2': 0, 'var3': 0},
    {'perk': 8226, 'var1': 250, 'var2': 2135, 'var3': 0},
    {'perk': 8210, 'var1': 8, 'var2': 0, 'var3': 0},
    {'perk': 8237, 'var1': 887, 'var2': 0, 'var3': 0}],
   'style': 8200},
  {'description': 'subStyle',
   'selections': [{'perk': 8345, 'var1': 3, 'var2': 0, 'var3': 0},
    {'perk': 8347, 'var1': 0, 'var2': 0, 'var3': 0}],
   'style': 8300}]}

In [7]:
 for key, value in matchs_data[0]["info"]["participants"][0]["perks"]["styles"][0].items():
        print(key, value, type(value))
        if type(value) is dict:
            print(key)
            

description primaryStyle <class 'str'>
selections [{'perk': 8230, 'var1': 18, 'var2': 0, 'var3': 0}, {'perk': 8226, 'var1': 250, 'var2': 2135, 'var3': 0}, {'perk': 8210, 'var1': 8, 'var2': 0, 'var3': 0}, {'perk': 8237, 'var1': 887, 'var2': 0, 'var3': 0}] <class 'list'>
style 8200 <class 'int'>


In [8]:
a = matchs_data[0]["info"]["participants"][0]
b = matchs_data[0]["info"]["participants"][1]
import copy
c = copy.deepcopy(a)
d = copy.deepcopy(b)
def popDict(dictionary,  base_dict):
    toRet = {}
    for key, value in base_dict.items():
        if type(value) is dict:
            nextdict = dictionary.pop(key, None)
            basenextdict = copy.deepcopy(nextdict)
            if type(nextdict) is dict:
                print("AAAAAAAAAAA")
                toRet.update(popDict(nextdict, basenextdict))
        elif type(value) is list:
            pass
        else:
            toRet[key] = value
    return toRet

"""
def extractFromDict(dictionary):
    toRet = {}
    for key, value in base_dict.items():
        if
        
        elif type(value) is list:
            for i in range
        else:
            toRet[key] = value

def checkElem(elem):
    if type(elem) is dict:
        extractFromDict(elem)
    elif type(elem) is list:
        for i in range(len(elem)):
"""           
dd = popDict(c, a)
tt = popDict(d, b)

AAAAAAAAAAA
AAAAAAAAAAA
AAAAAAAAAAA
AAAAAAAAAAA
AAAAAAAAAAA
AAAAAAAAAAA
AAAAAAAAAAA
AAAAAAAAAAA


In [9]:
print(type(d))
e = pd.DataFrame(tt, index=[0])
f = pd.DataFrame(dd, index=[0])

<class 'dict'>


In [10]:
e

,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,12AssistStreakCount,abilityUses,acesBefore15Minutes,alliedJungleMonsterKills,...,trueDamageTaken,turretKills,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,1,32,15,0,0,0,1,357,0,65,...,8935,0,11,0,0,40,0,3,18,False


In [11]:
g = e.join(f, lsuffix='1', rsuffix='2', how='outer').reset_index()
g

,index,allInPings1,assistMePings1,assists1,baronKills1,basicPings1,bountyLevel1,12AssistStreakCount1,abilityUses1,acesBefore15Minutes1,...,trueDamageTaken2,turretKills2,turretsLost2,unrealKills2,visionClearedPings2,visionScore2,visionWardsBoughtInGame2,wardsKilled2,wardsPlaced2,win2
0,0,1,32,15,0,0,0,1,357,0,...,5915,3,11,0,0,30,0,2,18,False


In [18]:
cure = matchs_data[0]["info"]["participants"][0]
cure.get("challenges")
#cuurr = cur.pop("challenges", None)
#cur
#pd.DataFrame(matchs_data[0]["info"]["participants"][0].get("challenges"))

{'12AssistStreakCount': 0,
 'abilityUses': 736,
 'acesBefore15Minutes': 0,
 'alliedJungleMonsterKills': 11,
 'baronTakedowns': 0,
 'blastConeOppositeOpponentCount': 0,
 'bountyGold': 150,
 'buffsStolen': 0,
 'completeSupportQuestInTime': 0,
 'controlWardsPlaced': 0,
 'damagePerMinute': 658.5782221076125,
 'damageTakenOnTeamPercentage': 0.24096884675262037,
 'dancedWithRiftHerald': 0,
 'deathsByEnemyChamps': 8,
 'dodgeSkillShotsSmallWindow': 2,
 'doubleAces': 0,
 'dragonTakedowns': 0,
 'earliestElderDragon': 2357.597933,
 'earlyLaningPhaseGoldExpAdvantage': 0,
 'effectiveHealAndShielding': 0,
 'elderDragonKillsWithOpposingSoul': 1,
 'elderDragonMultikills': 0,
 'enemyChampionImmobilizations': 96,
 'enemyJungleMonsterKills': 8,
 'epicMonsterKillsNearEnemyJungler': 0,
 'epicMonsterKillsWithin30SecondsOfSpawn': 0,
 'epicMonsterSteals': 0,
 'epicMonsterStolenWithoutSmite': 0,
 'firstTurretKilled': 1,
 'firstTurretKilledTime': 1055.5488375999998,
 'flawlessAces': 0,
 'fullTeamTakedown': 1,
 

In [19]:
pd.DataFrame.from_dict(matchs_data[0]["info"]["participants"][0])

,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,championId,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
12AssistStreakCount,0,8,11,0,0,0,0,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
abilityUses,0,8,11,0,0,0,736,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
acesBefore15Minutes,0,8,11,0,0,0,0,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
alliedJungleMonsterKills,0,8,11,0,0,0,11,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
baronTakedowns,0,8,11,0,0,0,0,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
playerScore7,0,8,11,0,0,0,NaN,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
playerScore8,0,8,11,0,0,0,NaN,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
playerScore9,0,8,11,0,0,0,NaN,24290,18,79,...,3,3,11,0,0,30,0,2,18,False
statPerks,0,8,11,0,0,0,NaN,24290,18,79,...,3,3,11,0,0,30,0,2,18,False


In [20]:
pd.options.display.max_columns = None
pd.DataFrame(matchs_data[0]["info"]["participants"][0]).columns

Index(['allInPings', 'assistMePings', 'assists', 'baronKills', 'basicPings',
       'bountyLevel', 'challenges', 'champExperience', 'champLevel',
       'championId',
       ...
       'turretKills', 'turretTakedowns', 'turretsLost', 'unrealKills',
       'visionClearedPings', 'visionScore', 'visionWardsBoughtInGame',
       'wardsKilled', 'wardsPlaced', 'win'],
      dtype='object', length=142)

In [21]:
pd.options.display.max_columns = None
a = pd.DataFrame(matchs_data[0]["info"]["participants"][0])[["allInPings", "assists", "summonerId"]]
b = pd.DataFrame(matchs_data[0]["info"]["participants"][1])[["allInPings", "assists", "summonerId"]]
c = pd.concat([a, b], axis=1)
c

,allInPings,assists,summonerId,allInPings,assists,summonerId
12AssistStreakCount,0.0,11.0,e3Q3ElRu6yp9w7HQCUiNgBWVsXXX_1QzesNsI5k77oLbYug,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
abilityUses,0.0,11.0,e3Q3ElRu6yp9w7HQCUiNgBWVsXXX_1QzesNsI5k77oLbYug,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
acesBefore15Minutes,0.0,11.0,e3Q3ElRu6yp9w7HQCUiNgBWVsXXX_1QzesNsI5k77oLbYug,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
alliedJungleMonsterKills,0.0,11.0,e3Q3ElRu6yp9w7HQCUiNgBWVsXXX_1QzesNsI5k77oLbYug,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
baronTakedowns,0.0,11.0,e3Q3ElRu6yp9w7HQCUiNgBWVsXXX_1QzesNsI5k77oLbYug,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
...,...,...,...,...,...,...
styles,0.0,11.0,e3Q3ElRu6yp9w7HQCUiNgBWVsXXX_1QzesNsI5k77oLbYug,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
earliestDragonTakedown,NaN,NaN,NaN,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
highestCrowdControlScore,NaN,NaN,NaN,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4
junglerKillsEarlyJungle,NaN,NaN,NaN,1.0,15.0,sSYQ8ZzqScje3ZPChToQUk-lEQBadAMlzgEEecZEymk0qT4


In [22]:
def get_matchids_by_puuid(puuid, num, api_key):

    matchids_list = []


    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count={}&api_key={}'.format(puuid,num, api_key)

    resp = requests.get(api_url)
    print(resp.status_code)
    if resp.status_code == 200: 
        pass
    elif resp.status_code == 429: #429 error is api cost issue
        while True: # while loop because of riot api cost
            if resp.status_code == 429: #429 error is api cost issue
                print('429 delay try 100 second') #approximate 110 second wait
                time.sleep(10)
                api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count={}&api_key={}'.format(puuid,num, api_key)
                resp = requests.get(api_url)
                print(resp.status_code)
            elif resp.status_code == 200:
                print('limit cost resolve')
                break        

    matchids = pd.DataFrame(resp.json(), columns = ['matchid']) 
    matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"

    matchids_df = pd.concat(matchids_list) 
    
    return matchids_df

In [23]:
def get_match_info_by_matchId(match_ids, api_key):

    matchid_info_list = [] # List that will store the dataframes generated, for each match id in the for loop 

    for i in range(len(match_ids)):# len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        api_ulr = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # here we use the iloc function as explained above to form the api link for each matchid
        resp = requests.get(api_ulr) # Sending a get request to the api_url
        print(resp.status_code) # Printing the status_code of the request

        if resp.status_code == 200: # Here we need to use if statment, because of the riot API rate limit which is 20 requests every 1 sec and 100 requests every 2 minutes. A 200 status_code means that the request was successful so it can move on.
            pass                        
        elif resp.status_code == 429: # A 429 means that we have exceeded the rate limit. To solve that we need to wait and then try again to get a successful request
            while True: # To do taht we use a while true loop, to execute the if, elif statment repeatdly until the status_code is 200. After that we use break statement to come out of this loop.
                if resp.status_code == 429: 
                    print('429 delay try 10 second') #approximate 110 second wait to work
                    time.sleep(10) # Sleep

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # Trying the request again
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break        

        # Same process we did in the beginning             
        matchid_info = pd.json_normalize(resp.json()['info']['participants'])
        matchid_info_list.append(matchid_info) # Merging all the dataframes generated and then stored in the matchid_info_list with the concat() function
        
    matchid_info_df = pd.concat(matchid_info_list)

    return matchid_info_df

In [24]:
matchsid = get_matchids_by_puuid(puuid, 10, api_key)
matchid_info_df= get_match_info_by_matchId(matchsid, api_key)


200
200
200
200
200
200
200
200
200
200
200


In [25]:
matchid_info_df

,allInPings,assistMePings,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,placement,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerScore0,playerScore1,playerScore10,playerScore11,playerScore2,playerScore3,playerScore4,playerScore5,playerScore6,playerScore7,playerScore8,playerScore9,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,riotIdGameName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.firstTurretKilledTime,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsOnRecentlyHealedByAramPack,challenges.killsUnderOwnTurret,challenges.killsWithHelpFromEpicMonster,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningP